<a href="https://colab.research.google.com/github/vrangayyan6/Machine-Learning/blob/main/SimpleTransformers_DistilBERT_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Oct 13 21:50:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#! pip uninstall torch torchvision -y
! pip install -Uqq torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install -Uqq transformers
! pip install -Uqq simpletransformers

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import pandas as pd

combine_path = "/content/drive/My Drive/Omdena/dataset_version2_commonsensemedia_reviews.csv"
input_df = pd.read_csv(combine_path, header=0)
print(input_df.shape)
input_df.head()

(40433, 17)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,game_name,review,user_name,reviewer_type,type,Positive Messages,Positive Role Models & Representations,Ease of Play,Violence,Sex,Language,Consumerism,"Drinking, Drugs & Smoking",review_sentiment,risk_level,risk_level_encoded,type_encoded
0,Flickr,Depends what photos you are looking at. If you...,ICEYWARM,Adult,website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,Flicker,high_risk,1,1
1,Flickr,"I hate it, its amazing what people say and pos...",2442hayhay2442,"Parent of a 8, 11, and 13-year-old",website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,age???,high_risk,1,1
2,Flickr,great site for all ages.,Kejr,Adult,website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,NaN,high_risk,1,1
3,Flickr,"Flickr can be a rough place, but I choose to s...",MisterObvious,"Teen, 14 years old",website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,For Photographers Only,high_risk,1,1
4,Flickr,"Flickr is a good photo sharing site, there's n...",TheBombFunn,"Teen, 13 years old",website,3.0,3.0,3.0,4.0,3.0,3.0,3.0,0.0,Great photo sharing website,high_risk,1,1


In [ ]:
combine_df = input_df.loc[:,["review", "Sex"]].rename(columns={"review": "Comments", "Sex": "label"})

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import numpy as np


stop_words = set(stopwords.words('english'))

def process_comments(comment):
  word_tokens = []
  if (type(comment)== str):
    word_tokens = [x.lower() for x in comment.split() if x.isalpha()]
  #comment = unicode(comment, 'utf8')
  tokens_stop = [word for word in word_tokens if (word not in stop_words)]
  # tokens_stop = [stemmer.stem(word) for word in word_tokens if (word not in stop_words)]
  comment_updated = []
  if len(tokens_stop) == 0:
    comment_updated = np.NaN
  else:
    comment_updated = ' '.join(tokens_stop)
  return comment_updated

combine_df['Comments'] = combine_df.apply(lambda row: process_comments(row['Comments']), axis=1)
print(combine_df.isnull().sum())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Comments    289
label       238
dtype: int64


In [ ]:
combine_df = combine_df.dropna().reset_index(drop=True)
print(combine_df.isnull().sum())

Comments    0
label       0
dtype: int64


In [ ]:
print(combine_df.shape)
print(combine_df['label'].value_counts())

(39915, 2)
0.0    18255
3.0    10609
1.0     3867
4.0     3223
2.0     2449
5.0     1512
Name: label, dtype: int64


In [ ]:
combine_df = combine_df.groupby('label').apply(lambda x: x.sample(n=1500, replace=True, random_state=10))

# max comment length
print(max(combine_df['Comments'].apply(len)))
combine_df.head()

4043


Comments  label
label                                                                
0.0   39120  sexual content violence hit players various we...    0.0
      18909  think five nights freddys good lot fun charact...    0.0
      39188  parents said sex language absolutely sexual co...    0.0
      18907  people say fnaf scary times blame jumpscares c...    0.0
      22163  nothing worry comes super mario series games d...    0.0

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(combine_df, test_size=0.2, random_state=10)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
import sklearn

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
'''
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 3,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}

'''

model_args = ClassificationArgs()
model_args.num_train_epochs = 3
model_args.max_seq_length = 512
# model_args.sliding_window = True
model_args.overwrite_output_dir = True
model_args.weight_decay = 0.01
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True

# Create a ClassificationModel
# You can set class weights by using the optional weight argument
model = ClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=6, args=model_args)

# Train the model
model.train_model(train_df, eval_df=val_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=sklearn.metrics.accuracy_score)

INFO:filelock:Lock 139636140291688 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock


INFO:filelock:Lock 139636140291688 released on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c.lock
INFO:filelock:Lock 139633329216144 acquired on /root/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


INFO:filelock:Lock 139633329216144 released on /root/.cache/torch/transformers/ae9df7a8d658c4f3e1917a471a8a21cf678fa1d4cb91e7702dfe0598dbdcf354.c2015533705b9dff680ae707e205a35e2860e8d148b45d35085419d74fe57ac5.lock


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

INFO:filelock:Lock 139633311536352 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.4693393708406514, 'eval_loss': 1.1548920503589841}


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.4671876107072592, 'eval_loss': 1.18968647049533}


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.486976426743962, 'eval_loss': 1.189856694440047}


INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.486976426743962, 'eval_loss': 1.189856694440047, 'acc': 0.5716666666666667}


In [ ]:
result

{'acc': 0.5716666666666667,
 'eval_loss': 1.189856694440047,
 'mcc': 0.486976426743962}

In [ ]:
# predict
# predictions, raw_outputs = model.predict(df_test.text.tolist())